# Restaurants from delivery sites & reviews

Restaurants are identified by names, similar names are not yet merged.  

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [2]:
import myfunctions as f
print f.today()

20180207


## 1. Restaurants from reviews
    attributes
>     1. Burpple review locs
    2. Food items mentioned in reviews: (sorted list of food_name & frequency mentioned in reviews)

In [3]:
review_path = "/home/yueliu/Desktop/workspace_yue"
file_name = "burpple__review__clean__20180205.p"    
file_name = os.path.join(review_path, file_name)
df_retrived = f.retrive_file(file_name)

retrived: /home/yueliu/Desktop/workspace_yue/burpple__review__clean__20180205.p


In [4]:
print df_retrived.shape
print df_retrived.columns.tolist()
df_retrived.head(1)

(323104, 10)
['loc', 'title', 'body', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls']


,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls
0,burppleinitial/review/--0EG2cp,Sicilian Lemon Creme andCaramelized Red Mixed ...,,OSO Ristorante,Sicilian Lemon Creme andCaramelized Red Mixed ...,"(#igsg, #igers, #igfood, #yum, #yummy, #instaf...",(),(),(),()


In [5]:
restaurants = df_retrived.groupby('restaurant.name')['loc'].apply(list).to_frame()
restaurants.reset_index(inplace=True)
restaurants.reset_index(inplace=True)
restaurants["index"] = restaurants["index"].apply(lambda s: "burpple_"+ str(s))

In [6]:
print "number of reataurants: %d"%restaurants.shape[0]

number of reataurants: 14455


In [7]:
# map loc to food items
# get a dictonary of {loc:[food_items]}

In [8]:
file_name = "review_with_synsets__20180207.p"
file_name = os.path.join(review_path, file_name)
loc_food = f.retrive_file(file_name)

retrived: /home/yueliu/Desktop/workspace_yue/review_with_synsets__20180207.p


In [9]:
from collections import defaultdict
loc_food_ref = defaultdict(set)
def get_loc_food_ref(line):
    food_item = line["food_items"]
    for loc in line["loc"]:
        loc_food_ref[loc].update([food_item])
loc_food.apply(get_loc_food_ref,1)
print "number of food review loc with food items: %d"%len(loc_food_ref)

number of food review loc with food items: 9540


In [10]:
def map_food(loc_lst):
    food_lst = []
    for loc in loc_lst:
        try:
            food_lst.append(list(loc_food_ref[loc]))
        except:
             pass 
    food_lst =  [item for sublist in food_lst  for item in sublist]
    return sorted([(x,food_lst.count(x)) for x in set(food_lst)], key=lambda x: (-x[1], x))

restaurants["restaurants.food"] = restaurants["loc"].apply(map_food)

In [11]:
restaurants.sample(5)

,index,restaurant.name,loc,restaurants.food
10179,burpple_10179,Saizeriya (Bukit Timah Plaza),"[burppleinitial/review/2zcgkF2e, burppleinitia...",[]
342,burpple_342,A-One Claypot House (ChinaTown Point),"[burppleinitial/review/4znBYLRw, burppleinitia...",[]
13178,burpple_13178,Tuk Tuk Cha (Tampines 1),"[burppleinitial/review/4G7I6XhE, burppleinitia...",[]
1281,burpple_1281,Bcasse Bakery (Dempsey Hill),"[burppleinitial/review/0a88NTZO, burppleinitia...","[(asparagus salad, 2), (apple tart, 1), (choic..."
1135,burpple_1135,Bali Thai (Causeway Point),"[burppleinitial/review/-jqUXJoA, burppleinitia...",[]


In [12]:
from copy import deepcopy
restaurant_entities = deepcopy(restaurants)

## 2. Restaurants from delivery sites
    attributes
>     1. Delivery sites locs
    2. Food items offered: (sorted list of food_name & count of delivery sites)

In [13]:
file_name = "all_food_vendor__standard__20180207.p"
#file_name = os.path.join(review_path, file_name)
df_retrived = f.retrive_file(file_name)

retrived: all_food_vendor__standard__20180207.p


In [14]:
print df_retrived.shape
print df_retrived.columns.tolist()
df_retrived.head(1)

(333572, 7)
['food_name', 'vendor_name', 'locs', 'clean_name', 'lemma_name', 'lemma_name_2', 'standard_name']


,food_name,vendor_name,locs,clean_name,lemma_name,lemma_name_2,standard_name
4,Japanese Pickle Roll (6 pcs),Sen of Japan,"[(2017-07-03T17:55:01.668428, deliveroo/food/7...",japanese pickle roll,japanese pickle roll,japanese pickle roll,japanese pickle roll


In [15]:
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
cols = ["vendor_name", "locs", "standard_name"]
df = df_retrived[cols]
df.rename(index=str, columns={"vendor_name": "restaurant.name"}, inplace=True)

def zip_food_loc(line):
    return [line["standard_name"],line["locs"]]
df["loc"] = df.apply(zip_food_loc, 1)

In [16]:
restaurants = df.groupby('restaurant.name')['loc'].apply(list).to_frame()
restaurants.reset_index(inplace=True)
restaurants.reset_index(inplace=True)
restaurants["index"] = restaurants["index"].apply(lambda s: "delivery_"+ str(s))

def count_food(lst):
    food_lst = []
    for p in lst:
        food_name = p[0]
        locs = p[1]
        delivery_sites = set()
        for loc in locs:
            delivery_sites.add(loc[1].split("/")[0])
        food_lst.append((food_name, len(delivery_sites)))
    return sorted(food_lst, key=lambda x: (-x[1], x))   
restaurants["restaurants.food"] = restaurants["loc"].apply(count_food)

In [17]:
# confirm indeed some food has more than 1 occurance
l = restaurants["restaurants.food"].tolist()
flat_list = [item for sublist in l for item in sublist]
selected = [i for i in flat_list if i[1]>1]
print "Some food is offered by some restaurant on multiple websites: %s"%(len(selected) > 0)

Some food is offered by some restaurant on multiple websites: True


In [18]:
restaurants.head(2)

,index,restaurant.name,loc,restaurants.food
0,delivery_0,,"[[trivento bird bee red mendoza argentina, [('...","[(aglio aglio, 1), (albarino ria baixas, 1), (..."
1,delivery_1,Aloha Pok,"[[aloha standard nalu tuna, [('2017-07-03T17:5...","[(aloha poke standard nalu salmon, 1), (aloha ..."


In [19]:
# extract loc to be list
def extract_loc(lst):
    all_locs = []
    for food in lst:
        food_locs = food[1]
        for l in food_locs:
            all_locs.append(l[1])
    return all_locs
restaurants["loc"] = restaurants["loc"].apply(extract_loc)

In [20]:
restaurants.sample(5)

,index,restaurant.name,loc,restaurants.food
5078,delivery_5078,Sushi Burrito,"[deliveroo/food/140462, deliveroo/food/AV07Ceb...","[(chawan mushi, 2), (cherry tomato salad, 2), ..."
4024,delivery_4024,Pho4u Halal Vietnamese Cuisine,"[foodpanda/menu_item/AV44BjegW4PLiMnQOVSi, foo...","[(banh mi, 1), (canh ga chien, 1), (cha gio, 1..."
541,delivery_541,Barrio by Mex Out - 313 @Somerset,"[deliveroo/food/AWCwWjstvFWOHWF15iyj, delivero...","[(barbacoa quesadilla, 1), (burn nachos, 1), (..."
847,delivery_847,Camellia Chinese Bistro,"[deliveroo/food/69271, deliveroo/food/AV0620rG...","[(bake cartilage with special sauce, 1), (brai..."
2800,delivery_2800,Krispy Kreme (Suntec City),"[foodpanda/menu_item/AV435W4_W4PLiMnQOJtB, foo...","[(chocolate cookie crunch, 1), (chocolate dip ..."


## 3. Merge records

In [21]:
restaurant_entities = pd.concat([restaurant_entities, restaurants],ignore_index=True)

In [22]:
restaurant_entities["food_cnt"] = restaurant_entities["restaurants.food"].apply(len)

In [23]:
print "%d restaurant entities with %d names"%(restaurant_entities.shape[0],
                                              restaurant_entities["restaurant.name"].nunique())
print "%d restaurant entities with food items"%(restaurant_entities[
    restaurant_entities["food_cnt"]>0].shape[0])

20857 restaurant entities with 20013 names
10095 restaurant entities with food items


In [24]:
restaurant_entities.sample(5)

,index,restaurant.name,loc,restaurants.food,food_cnt
14415,burpple_14415,somerset 313,[burppleinitial/review/NTIOtUBk],[],0
2830,burpple_2830,Cookwerx by Chef Taj,"[burppleinitial/review/CZXGtB-5, burppleinitia...",[],0
2535,burpple_2535,Chocolate Atelier,[burppleinitial/review/LHf6MVOl],"[(and, 1), (flavour, 1), (for, 1)]",3
14764,delivery_309,AlibabaR - Yong Huat (Since 1949),"[deliveroo/food/62570, deliveroo/food/AV061swm...","[(fishball mince meat noodle dry, 1), (fishbal...",18
15655,delivery_1200,Craftsmen - Tanah Merah,"[deliveroo/food/136008, deliveroo/food/AV07BsS...","[(back bacon, 1), (bake bean, 1), (butter choc...",45


In [25]:
file_name = "__".join(["restaurant_entities", f.today()]) + ".p"
f.save_file(restaurant_entities, file_name)

saved: restaurant_entities__20180207.p
